In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionnConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from BriefAI.constants import *
from BriefAI.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionnConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionnConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [6]:
import os 
import urllib.request as request
import zipfile
from BriefAI.logging import logger
from BriefAI.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self,config: DataIngestionnConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory 
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

2025-08-28 11:12:19,123 - BriefAI Logger - INFO - yaml file: config/config.yaml loaded successfully
2025-08-28 11:12:19,125 - BriefAI Logger - INFO - yaml file: params.yaml loaded successfully
2025-08-28 11:12:19,126 - BriefAI Logger - INFO - created directory at: artifacts
2025-08-28 11:12:19,127 - BriefAI Logger - INFO - created directory at: artifacts/data_ingestion
2025-08-28 11:12:22,730 - BriefAI Logger - INFO - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 8377572
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3aea79eaa36efc9339c42d3a61413f75f62f7e5676002d7473efbe842605ace6"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DD7E:2F7C73:51F1A:FF8AB:68AFEC38
Accept-Ranges: bytes
Date: Thu, 28 Aug 2025 05:42:21 GMT
Via: 1.1 varnish
X-